<a href="https://colab.research.google.com/github/Jaat2727/Torrent-Downloader-on-Collab-To-Google-Drive/blob/main/Best_Simple_TORRENT_DOWNLOADER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 📂 Install Libraries & Mount Google Drive
#@markdown Run this cell to:
#@markdown 1. Install necessary Python libraries (**torrentp**, **nest_asyncio**).
#@markdown 2. Connect and mount your Google Drive to `/content/drive`.
#@markdown You might be asked to authorize Drive access.

import os
import sys
import subprocess
import time

print("--- Starting Setup ---", flush=True)
setup_ok = True

# 1. Install Libraries (torrentp and nest_asyncio)
print("\n⚙️ Installing/checking required libraries (torrentp, nest_asyncio)...", flush=True)
try:
    # Use -q for quiet, capture_output to check errors if needed
    install_process = subprocess.run(
        [sys.executable, '-m', 'pip', 'install', 'torrentp', 'nest_asyncio', '-q'],
        check=True,
        capture_output=True,
        text=True
    )
    # Check if installation actually happened or if they were already there
    # This is a basic check; pip's output could be parsed more thoroughly if needed.
    # if install_process.stdout:
    #      print(f"   Pip output:\n{install_process.stdout[:500]}...") # Show some output if needed
    print("✅ Libraries installed or already present.")
except subprocess.CalledProcessError as e:
    print(f"❌ ERROR installing libraries: Pip command failed.", flush=True)
    print(f"   Stderr: {e.stderr}")
    setup_ok = False
except Exception as e:
    print(f"❌ ERROR during library installation: {e}", flush=True)
    setup_ok = False

# 2. Mount Google Drive (only if library install was ok)
if setup_ok:
    print("\n⚙️ Mounting Google Drive...", flush=True)
    try:
        from google.colab import drive
        if not os.path.exists('/content/drive/MyDrive'):
            drive.mount('/content/drive')
            # Add a small delay for the mount point to become fully available
            time.sleep(2)
            if os.path.exists('/content/drive/MyDrive'):
                 print("✅ Google Drive mounted successfully.")
            else:
                 print("⚠️ Mount attempt finished, but '/content/drive/MyDrive' not found. Check authorization/permissions.", flush=True)
                 # Depending on strictness, could set setup_ok = False here
        else:
            print("✅ Google Drive already mounted.")
    except Exception as e:
        print(f"❌ An error occurred during mounting: {e}", flush=True)
        setup_ok = False
else:
    print("\n⚠️ Skipping Google Drive mount due to library installation failure.", flush=True)

# 3. Final Status
print("\n--- Setup Finished ---", flush=True)
if setup_ok:
    print("✅ Setup complete. You can now run the Torrent Downloader cell.", flush=True)
else:
    print("❌ Setup failed. Please review the errors above.", flush=True)

.

.

.

.

In [ ]:
#@title 🧲 Google Drive Torrent Downloader
#@markdown Enter the magnet link and download path below.
#@markdown ---
#@markdown **Enter Magnet Link:**
MAGNET_LINK = "your  magnet_link  place it here" #@param {type:"string"}

#@markdown **Settings:**
DOWNLOAD_PATH = "/content/drive/MyDrive/MyDownloads" #@param {type:"string"}
#@markdown *(Make sure Google Drive is mounted at /content/drive) and if you dont know what to add in path just leave as it is*
#@markdown ---

import asyncio
import nest_asyncio
from torrentp import TorrentDownloader
import time
import os
import threading
import traceback # Added for potential error details

# Enable nested asyncio (REQUIRED for Colab)
try:
    nest_asyncio.apply()
except RuntimeError: # Already applied
    pass

# --- Keep-Alive (Prevent Timeout) ---
# Note: Ensure only one keep-alive thread runs if executing multiple cells
keep_alive_active_orig = threading.Event()
def keep_alive_orig():
    while not keep_alive_active_orig.wait(60):
        print("Keep-alive (Original): Simulating activity...", flush=True)
    print("Keep-alive (Original): Thread stopping.")

keep_alive_thread_orig = threading.Thread(target=keep_alive_orig, name="ColabKeepAliveOrig", daemon=True)
if not any(t.name == "ColabKeepAliveOrig" for t in threading.enumerate()):
     keep_alive_thread_orig.start()
# --- End Keep-Alive ---

async def monitor_download(torrent_file, interval=5):
    """Monitors download progress and prints statistics."""
    # !!! WARNING: torrent_file.is_completed() likely does not exist and will error !!!
    while not torrent_file.is_completed(): # <--- LIKELY ERROR SOURCE
        try:
            stats = torrent_file.get_stats()
            total_size_gb = stats.total_size / (1024**3) if stats.total_size else 0
            downloaded_gb = stats.downloaded / (1024**3) if stats.downloaded else 0
            progress = (stats.downloaded / stats.total_size) * 100 if stats.total_size else 0
            download_speed_mbps = (stats.download_speed / (1024**2)) * 8 if stats.download_speed else 0
            upload_speed_mbps = (stats.upload_speed / (1024**2)) * 8 if stats.upload_speed else 0
            num_peers = stats.num_peers

            print(
                f"Progress: {progress:.2f}% | "
                f"Downloaded: {downloaded_gb:.2f} GB / {total_size_gb:.2f} GB | "
                f"Down Speed: {download_speed_mbps:.2f} Mbps | "
                f"Up Speed: {upload_speed_mbps:.2f} Mbps | "
                f"Peers: {num_peers}",
                flush=True
            )
            await asyncio.sleep(interval)
        except asyncio.CancelledError:
             print("Monitor cancelled.", flush=True)
             break # Exit loop if cancelled
        except AttributeError as ae:
             # Catch the likely error and stop monitoring
             print(f"\n❌ MONITORING ERROR: {ae}", flush=True)
             print("   Stopping monitoring due to likely invalid method call.", flush=True)
             break
        except Exception as e:
             print(f"\nError during monitoring: {e}", flush=True)
             await asyncio.sleep(interval) # Wait before retrying stats


    # This part will likely not be reached if is_completed() fails
    print("Monitoring loop finished.", flush=True)
    # !!! WARNING: torrent_file.get_downloaded_files() likely does not exist and will error !!!
    # print(torrent_file.get_downloaded_files()) # <--- LIKELY ERROR SOURCE


async def main(magnet_input, download_path_input):
    """Downloads a torrent from a magnet link, with monitoring and Colab-specific handling."""
    torrent_file = None
    monitor_task = None
    download_task = None
    try:
        # --- Input Validation ---
        if not magnet_input or not magnet_input.startswith("magnet:?"):
            print("❌ Invalid or empty Magnet Link provided."); return
        if not download_path_input:
            print("❌ Download path is empty."); return
        if not os.path.exists('/content/drive/MyDrive'):
            print("❌ Google Drive not mounted at /content/drive/MyDrive."); return
        try: os.makedirs(download_path_input, exist_ok=True)
        except OSError as e: print(f"❌ ERROR: Cannot access base directory '{download_path_input}': {e}"); return

        # --- Start Process ---
        print(f"Initializing torrent: {magnet_input[:60]}...", flush=True)
        torrent_file = TorrentDownloader(magnet_input, download_path_input)
        print(f"Starting download to {download_path_input}...", flush=True)

        # Run download and monitoring concurrently
        download_task = asyncio.create_task(torrent_file.start_download(), name="OriginalDownload")
        # Pass display name, fallback to 'Unknown'
        display_name = getattr(torrent_file, 'name', 'Unknown Torrent') # Try to get name if available
        monitor_task = asyncio.create_task(monitor_download(torrent_file, display_name), name="OriginalMonitor")

        # Wait for download task to complete (or be interrupted/error)
        print("Waiting for download task to complete...", flush=True)
        await download_task
        print("Download task finished.", flush=True)

        # Download finished successfully (if await download_task didn't raise error)
        # Now explicitly handle monitor cancellation as in original code
        if monitor_task and not monitor_task.done():
            print("Cancelling the monitoring task...", flush=True)
            monitor_task.cancel()
            try:
                await monitor_task  # Ensure cancellation is complete
            except asyncio.CancelledError:
                print("Monitoring task successfully cancelled.", flush=True)
            except Exception as e_cancel:
                 print(f"Error awaiting cancelled monitor task: {e_cancel}", flush=True)

    except Exception as e:
        print(f"\n❌ An error occurred in main: {type(e).__name__}: {e}", flush=True)
        traceback.print_exc() # Print detailed traceback
        # Cancel tasks if they are running on error
        if monitor_task and not monitor_task.done(): monitor_task.cancel()
        if download_task and not download_task.done(): download_task.cancel()
        # Await cancellations
        all_tasks_err = [t for t in [download_task, monitor_task] if t]
        if all_tasks_err: await asyncio.gather(*all_tasks_err, return_exceptions=True)

    finally:
        print("\n--- Main Coroutine Finally Block ---", flush=True)
        if torrent_file:
            print("   Stopping download (calling stop_download)...", flush=True)
            try:
                torrent_file.stop_download()
                print("   stop_download called.")
            except Exception as e_stop:
                 print(f"   Error calling stop_download: {e_stop}", flush=True)
        # Signal keep-alive thread to stop
        print("   Signalling keep-alive thread to stop.", flush=True)
        keep_alive_active_orig.set()
        print("------------------------------------", flush=True)

# --- Execution Block ---
# This runs when the cell is executed after filling the form
if not MAGNET_LINK:
    print("⚠️ Please enter a Magnet Link in the form above.", flush=True)
elif not os.path.exists('/content/drive/MyDrive'):
     print("❌ Google Drive does not appear to be mounted. Please run the mounting cell first.", flush=True)
else:
    print("🚀 Starting Original Download Process...", flush=True)
    loop = None
    try:
        loop = asyncio.get_event_loop()
        print("   Running asyncio loop until main coroutine completes.", flush=True)
        # Call the main async function with values from the form
        loop.run_until_complete(main(MAGNET_LINK, DOWNLOAD_PATH))
        print("🏁 Asyncio main coroutine finished.", flush=True)
    except Exception as loop_err:
        print(f"\n❌❌❌ Unexpected error running the asyncio loop: {loop_err}", flush=True)
        traceback.print_exc()
    finally:
        if not keep_alive_active_orig.is_set(): # Ensure stop signal even on loop error
            keep_alive_active_orig.set()

    print("ℹ️ Cell execution logic complete.", flush=True)

# --- Final Cell Message ---
if not os.path.exists('/content/drive/MyDrive') and MAGNET_LINK: print("\n--- End of Cell Execution (Drive Not Mounted) ---")
elif not MAGNET_LINK: print("\n--- End of Cell Execution (No Magnet Link Provided) ---")
else: print("\n--- End of Cell Execution ---")

.

  .

.

.

.

In [ ]:
#@title 🚀 Open Google Drive in a New Tab (Attempt Auto-Open)
#@markdown Click Here and then click on output link to open your Google Drive

from IPython.display import display, HTML

drive_url = "https://drive.google.com/drive/u/0/my-drive" # The standard Google Drive URL
html_code = f'''
<a href="{drive_url}" target="_blank">Click here to open your Google Drive (if it doesn't open automatically)</a>
'''
display(HTML(html_code))